In [1]:
import numpy as np
from scipy import signal

import math

import matplotlib.pyplot as plt
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

TODO: 
- test

In this notebook, we simulate a rough version of the SABR-Local Stochastic Volatility model, which is given by
$$
    \begin{cases}
        dS_t = S_t L(t, S_t) V^\alpha_t dW_t \\
        dV_t = \nu V_t dB_t \\
        V^\alpha_t = \mathcal{G^\alpha}\{V_t\} \\
        d\langle W, B \rangle_t = \rho dt
    \end{cases}
$$
with
$$
    \mathcal{G^\alpha}\{f\}(t) := \begin{cases} \int_0^t f(s) \frac{d}{dt}g(t-s)ds, \quad \text{if } \alpha \in [0, \frac{1}{2}) \\
    \frac{d}{dt}\int_0^t f(s)g(t-s)ds, \quad \text{if } \alpha \in (-\frac{1}{2}, 0)
    \end{cases}
$$

In [17]:
n = 1000 # Number of timesteps
sqrtn = math.sqrt(n) 
M = 1 # Number of simulations
rho = 1 # Correlation between the normal distributions dzeta and eta
T = 1 # Total time

# Some volatility parameters
vola_0 = 1 # initial value of the non-rough vola
nu = 1
alpha = -0.25 # Alpha as in G^\alpha. It has to lie in (-.5, .5) to have any meaning. For negative alphas, you get roughness. 'equivalent' to fBM with H = alpha + .5
X_0 = 1 # initial value of X, the log-stock price 

In [3]:
def g(t):
    return t**alpha
g_vector = np.fromfunction(lambda i: g(T*i/n), shape=(n,), dtype=int)
g_vector[0] = 0

C:\Users\milog\Anaconda3\envs\MLIF Project\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in power
  


In [12]:
def b(y):
    return 0
def a(y):
    return nu*y
def Phi(y):
    return y**2

In [40]:
vola = np.zeros((n,)) # The discretised non-rough volatility process
Delta_vola = np.zeros((n,)) # non-rough volatility increments vola_i - vola_{i-1}

The system is ran according to
$$
\begin{cases} 
dX_t = V_t^\alpha L(t, X_t) dW_t - \frac{1}{2}(V_t^\alpha)^2 L^2(t, X_t)dt \\
dV_t = \nu V_t dB_t \\
V_t^\alpha = \mathcal{G}^\alpha\{V_t\} \\
d\langle W, B \rangle_t = \rho dt
\end{cases}
$$
Where $X := \log(S)$ is the log stock price and L is the leverage function, see see <cite data-cite="horvath2017functional">Horvath, Blanka, Antoine Jack Jacquier, and Aitor Muguruza. "Functional central limit theorems for rough volatility." Available at SSRN 3078743 (2017)</cite>

In [42]:
W = np.random.normal(0.0, 1.0, size=(n,)) # The unscaled brownian motion increments
vola[0] = vola_0
for i in range(1, n):
    vola[i] = vola[i-1] + (T/n)*b(vola[i-1]) + (T/sqrtn)*a(vola[i-1])*W[i-1]
    Delta_vola[i] = vola[i] - vola[i - 1]

In [19]:
rough_vola = signal.fftconvolve(g_vector, Delta_vola, 'same')

In [20]:
X = np.zeros((M, n))
for j in range(M):
    # Create correlated Brownian motion
    randoms = np.random.normal(0.0, 1.0, size=(n,))
    B = rho*W + math.sqrt(1 - rho**2) * randoms
    X[j, 0] = X_0
    for i in range(1, n):
        X[j, i] = X[j, i-1] - (1/2)*(T/n)*Phi(rough_vola[i-1]) + math.sqrt((T/n) * Phi(rough_vola[i-1])) * B[i-1]

In [46]:
# A function which gives the brownian motion used to generate the rough volatility and the volatility.
def rough_volatility():
    vola = np.zeros((n,))
    Delta_vola = np.zeros((n,))
    W = np.random.normal(0.0, 1.0, size=(n,))
    g_vector = np.fromfunction(lambda i: g(T*i/n), shape=(n,), dtype=int)
    vola[0] = vola_0
    for i in range(1, n):
        vola[i] = vola[i-1] + (T/n)*b(vola[i-1]) + (T/sqrtn)*a(vola[i-1])*W[i-1]
        Delta_vola[i] = vola[i] - vola[i - 1]
    rough_vola = signal.fftconvolve(g_vector, Delta_vola, 'same')
    return (W, G_alpha_Y)